## Find scenes for selected path/row

In [1]:
from pystac_client import Client

# Search against the Planetary Computer STAC API
catalog = Client.open(
  "https://planetarycomputer.microsoft.com/api/stac/v1"
)

# Define your area of interest
aoi = {
  "type": "Polygon",
  "coordinates": [
    [
      [54.6355092639692, 42.68420851174815],
      [63.69846542906245, 42.68420851174815],
      [63.69846542906245, 47.318581705920565],
      [54.6355092639692, 47.318581705920565],
      [54.6355092639692, 42.68420851174815]
    ]
  ]
}

# Define your temporal range
daterange = {"interval": ["1982-08-22T00:00:00Z", "2023-06-21T23:59:59Z"]}

# Define your search with CQL2 syntax
search = catalog.search(filter_lang="cql2-json", filter={
  "op": "and",
  "args": [
    {"op": "s_intersects", "args": [{"property": "geometry"}, aoi]},
    {"op": "anyinteracts", "args": [{"property": "datetime"}, daterange]},
    {"op": "=", "args": [{"property": "collection"}, "landsat-c2-l2"]},
    {"op": "in", "args": [{"property": "landsat:wrs_path"}, ["160", "161"]]},
    {"op": "in", "args": [{"property": "landsat:wrs_row"}, ["028", "029"]]},
    {"op": "<=", "args": [{"property": "eo:cloud_cover"}, 5]}
  ]
})

## Extract scene IDs

In [2]:
items = [item.assets["red"].href.split("/")[-1].strip("_B4.TIF").strip("_B3.TIF") for item in search.get_items()]
len(items)

1434

## Generate collection dict

In [3]:
LAKE_NAME = "the Aral Sea"
LAKE_NAME_SLUG = "aral-sea"

In [4]:
import datetime

xmin = ymin = 1000
xmax = ymax = -1000

mindate = datetime.datetime(2100, 1, 1, tzinfo=datetime.timezone.utc)
maxdate = datetime.datetime(1900, 1, 1, tzinfo=datetime.timezone.utc)

for item in search.get_items():
    xmin = min(xmin, item.bbox[0])
    ymin = min(ymin, item.bbox[1])
    xmax = max(xmax, item.bbox[2])
    ymax = max(xmax, item.bbox[3])
    
    mindate = min(mindate, item.datetime)
    maxdate = max(maxdate, item.datetime)

BBOX = (xmin, ymin, xmax, ymax)
print(BBOX)

MIN_DATE = mindate
MAX_DATE = maxdate

COLLECTION_INTERVAL = [d.strftime("%Y-%m-%dT%H:%M:%S") + "Z" for d in (mindate, maxdate)]
print(COLLECTION_INTERVAL)

(56.91258402, 43.48369514, 63.64131735, 63.64131735)
['1986-04-10T06:12:19Z', '2023-06-20T06:40:19Z']


In [5]:
collection = {
  "id": f"landsat-c2l2-sr-lakes-{LAKE_NAME_SLUG}",
  "type": "Collection",
  "title": f"Selected Landsat 7 through 9 Surface Reflectance Scenes for {LAKE_NAME}",
  "extent": {
    "spatial": {
      "bbox": BBOX
    },
    "temporal": {
      "interval": [
        [
          f"{MIN_DATE:%Y-%m-%dT%H:%M:%S}Z",
          f"{MAX_DATE:%Y-%m-%dT%H:%M:%S}Z"
        ]
      ]
    }
  },
  "license": "https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/Landsat_Data_Policy.pdf",
  "summaries": {
    "datetime": [
    ],
    "cog_default": {
      "max": None,
      "min": None
    }
  },
  "description": f"Selected low-cloud-cover Landsat Collection-2 Level-2 Surface Reflectance scenes covering {LAKE_NAME}. Products from multiple missions and instruments are combined in this single collection for the purpose of visualizing RGB images. Note that the full item records are not interchangeable across instruments for purposes other than visualization and that the assets available are not the same for each instrument.",
  "item_assets": {
    "red": {
      "type": "image/vnd.stac.geotiff; cloud-optimized=true",
      "roles": [
        "data"
      ],
      "title": "Red Band (B4)",
      "eo:bands": [
        {
          "gsd": 30,
          "name": "B4",
          "common_name": "red",
          "center_wavelength": 0.65
        }
      ],
      "description": "Collection 2 Level-2 Red Band (B4) Surface Reflectance"
    },
    "blue": {
      "type": "image/vnd.stac.geotiff; cloud-optimized=true",
      "roles": [
        "data"
      ],
      "title": "Blue Band (B2)",
      "eo:bands": [
        {
          "gsd": 30,
          "name": "B2",
          "common_name": "blue",
          "center_wavelength": 0.48
        }
      ],
      "description": "Collection 2 Level-2 Blue Band (B2) Surface Reflectance"
    },
    "green": {
      "type": "image/vnd.stac.geotiff; cloud-optimized=true",
      "roles": [
        "data"
      ],
      "title": "Green Band (B3)",
      "eo:bands": [
        {
          "gsd": 30,
          "name": "B3",
          "common_name": "green",
          "center_wavelength": 0.56
        }
      ],
      "description": "Collection 2 Level-2 Green Band (B3) Surface Reflectance"
    },
    "thumbnail": {
      "type": "image/jpeg",
      "roles": [
        "thumbnail"
      ],
      "title": "Thumbnail image"
    },
    "cog_default": {
      "type": "image/tiff; application=geotiff; profile=cloud-optimized",
      "roles": [
        "data",
        "layer"
      ],
      "title": "Default COG Layer",
      "description": "Cloud optimized default layer to display on map"
    }
  },
  "stac_version": "1.0.0",
  "dashboard:is_periodic": "false",
  "dashboard:time_density": "day"
}

## Write to files

In [6]:
import json

with open(f"{LAKE_NAME_SLUG}-scenes.json", "w") as fobj:
    json.dump(items, fobj, indent=2)
    
with open(f"{LAKE_NAME_SLUG}-collection.json", "w") as fobj:
    json.dump(collection, fobj, indent=2)